# Лабораторная работа №1, Ансамбли моделей

## Выполнил: Лабуткин Иван Алексеевич, группа 4144

## Импорт библиотек

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import mode
from RandomForest import cross_validation, RandomForest, accuracy
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

## Чтение и предобработка данных

In [3]:
data=pd.read_csv('breast-cancer.csv', sep=',')

In [4]:
data=data.drop(['id'], axis=1)
y=data[['diagnosis']].to_numpy().reshape(-1)
data=data.drop(['diagnosis'], axis=1)
data=data.drop(['radius_mean', 'area_mean', 'radius_mean', 'radius_worst', 'area_worst', 'perimeter_worst'], axis=1)
data=data.drop(['concavity_mean', 'concave points_mean', 'texture_worst'], axis=1)
data=data.drop(['area_se', 'perimeter_se', 'compactness_worst','concave points_worst'], axis=1)
data=data.drop(['smoothness_worst','concavity_worst'], axis=1)
X=data.to_numpy()

In [5]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3, random_state=24)

## Обучение модели и сравнение с эталонной реализацией

In [6]:
rf=RandomForest(n_trees=300, max_depth=5, n_features=4,min_samples_leaf=2,min_samples_split=2)

In [225]:
rf.fit(X_train,y_train)

In [227]:
y_pred=rf.predict(X_test)

In [229]:
print('Точность написанного алгоритма на тестовой выборке:', accuracy(y_pred,y_test))
print('Точность написанного алгоритма на тренировочной выборке:', accuracy(rf.predict(X_train),y_train))

Точность написанного алгоритма на тестовой выборке: 0.935672514619883
Точность написанного алгоритма на тренировочной выборке: 0.9874371859296482


In [231]:
clf=RandomForestClassifier(max_depth=5, random_state=0, criterion='entropy', n_estimators=300, min_samples_leaf=2,min_samples_split=2)

In [233]:
clf.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=5, min_samples_leaf=2,
                       n_estimators=300, random_state=0)

In [235]:
y_pred=clf.predict(X_test)

In [239]:
print('Точность эталонной реализации на тестовой выборке:', accuracy(y_pred,y_test))

Точность эталонной реализации на тестовой выборке: 0.935672514619883


Точности идентичны. Можно обучить на сетке эталонный алгоритм и получить оптимальные гиперпараметры

In [147]:
grid_space={'max_depth':[3,5,7,10],
              'n_estimators':[10*i for i in range(2,21)],
              'min_samples_leaf':[1,2,5,8,10],
              'min_samples_split':[2,5,8,10],
            'criterion':['entropy','gini']
           }

grid = GridSearchCV(clf,param_grid=grid_space,cv=5,scoring='accuracy')
model_grid = grid.fit(X,y)

In [148]:
model_grid.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 130}

In [9]:
rf=RandomForest(n_trees=130, max_depth=10, n_features=4,min_samples_leaf=2,min_samples_split=5, eps_1=0.55, eps_2=0.55)

In [11]:
start = time.time()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
end = time.time()

In [246]:
print("Время выполнения программы:",
      (end-start) * 10**3, "ms")

Время выполнения программы: 83.26458930969238 ms


In [299]:
clf=RandomForestClassifier(max_depth=10, criterion='entropy', n_estimators=130, min_samples_leaf=2,min_samples_split=5)

In [301]:
start = time.time()
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
end = time.time()

In [303]:
print("Время выполнения программы:",
      (end-start) * 10**3, "ms")

Время выполнения программы: 122.92647361755371 ms


Написанная вручную реализация оказалась быстрее

Оценим точность алгоритма с помощью кросс-валидации:

In [10]:
print("Точность написанного вручную алгоритма с использованием кросс-валидации:", cross_validation(rf,X,y,5))

Точность написанного вручную алгоритма с использованием кросс-валидации: 0.936686849868033


In [319]:
print("Точность эталонной реализации с использованием кросс-валидации:", cross_validation(rf,X,y,5))

Точность эталонной реализации с использованием кросс-валидации: 0.9367179009470579


Результаты оказались практически идентичными